In [1]:
import os
os.environ["LD_LIBRARY_PATH"] = os.environ["LD_LIBRARY_PATH"] + ":'/app/distrib_manual/jupiter/venv/lib64/python3.9/site-packages/pysqlite3_binary.libs"


In [2]:
import sys
import json
from tqdm import tqdm
import os
from functools import reduce
from typing import Dict
import gc
from gigachat import GigaChat 
from gigachat.models import Chat, Messages

# TO CHANGE
BASEDIR = os.path.abspath(os.getcwd() + "/../")
__import__('pysqlite3')
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
# TO CHNAGE
os.environ["LD_LIBRARY_PATH"] = BASEDIR + "/lib"
sys.path.insert(0, BASEDIR)

from src.memorize_pipeline import MemPipeline
from src.memorize_pipeline.extractor.LLMExtractor import LLMExtractor
from src.memorize_pipeline.updator.LLMUpdator import LLMUpdator
from src.agents.hosted.agent_connector import AgentConnector
from src.utils.data_structs import TripletCreator, NodeCreator, Relation, NODES_TYPES_MAP, RELATIONS_TYPES_MAP
from src.agents.utils import SYSTEM_PROMPT

from src.neo4j_functions import Neo4jConnection
from src.embedding_functions import EmbeddingsDatabaseConnection, EmbeddingsDatabaseConnectionConfig, VectorDBConnectionConfig
from src.knowledge_graph_model import KnowledgeGraphModel

DATASET_PATH = '../data/Augment_DiaASQ.json'
LOAD_EXTRACTED_TRIPLETS_FILE = ''
gc.collect()

1107

In [3]:
class GigaChatAgent:
    def __init__(self, creds: str, scope: str = 'GIGACHAT_API_CORP', model: str = "GigaChat-Pro",
                 verify_ssl_certs: bool = False) -> None:
        # self.giga_model = GigaChat(
        #     credentials=creds, scope=scope, verify_ssl_certs=verify_ssl_certs, model=model) 
        self.giga_model = GigaChat(base_url="https://gigachat-ift.sberdevices.delta.sbrf.ru/v1/",
                                   scope = "GIGACHAT_API_CORP", 
                                   cert_file="/app/certs/giga/published.pem",
                                   key_file="/app/certs/giga/00CA0001CLLMllm99usr.key", 
                                   verify_ssl_certs=False,
                                   timeout=3600,)
        self.system_prompt = SYSTEM_PROMPT

    def generate(self, user_prompt: str, assistant_prompt: str = None, 
                 system_prompt: str = None, gen_strategy: Dict = None) -> str:

        chat = Chat(messages=[Messages(role='system', content=system_prompt if system_prompt is not None else self.system_prompt), 
                            Messages(role='user', content=user_prompt)]) 
        response = self.giga_model.chat(chat) 
        return response.choices[0].message.content

In [4]:
API_KEY = "OWUwOGUzOWEtMjJiNi00YmMxLThmMmItNzMwNjM2MTI2YmYxOjg2ODdiOTVhLTZkNDctNGFjOC1iMmViLTEyNDA5MmFiN2Q5Mw=="
agent = GigaChatAgent(creds=API_KEY)

In [5]:
agent.generate("Сколько будет 2+2?")

'2 + 2 равно 4.'

### Extract

In [6]:
DATASET_PATH

'../data/Augment_DiaASQ.json'

In [6]:
with open(DATASET_PATH, 'r', encoding='utf-8') as fd:
    data = json.loads(fd.read())

In [9]:
with open("./doc_data.jsonl", 'r', encoding='utf-8') as fd:
    # data = json.loads(fd.read())
    data = [json.loads(line) for line in fd]

In [10]:
data

[{'doc_name': 'doc_5.pdf',
  'page': '0',
  'text': 'Государственная  \nавтоматизированная  \nинформационная  \nсистема  \n«ЭРА-ГЛОНАСС» \n'},
 {'doc_name': 'doc_5.pdf',
  'page': '1',
  'text': 'с 1 января 2015 года функционирует \n«ЭРА-ГЛОНАСС» - первая в мире национальная система экстренного  реагирования при дорожно-транспортных происшествиях \n2 \nСистема/страна Главная цель и охват системы Сроки ввода \nЭРА-ГЛОНАСС \neCall \nSIMRAV \nEDR \nЭВАК \nЭРА-РБ \nПомощь при ДТП - новые типы транспортных средств. С 1 января 2017 года оснащаются все  новые автомобили. \nПомощь при ДТП - весь автотранспорт \nОхрана и поиск - весь автотранспорт\u200b \n«Черные ящики» - весь автотранспорт\u200b \nПомощь при ДТП - новые автомобили\u200b \nПомощь при ДТП - новые автомобили\u200b \nразрозненные  внедрения по странам  ЕС (27 стран) \nс 2017 года - частично \nс 2017 года – в  статусе «пилотного  проекта»\u200b \nс 1 января 2019 года функционирует \nна этапе создания \n'},
 {'doc_name': 'doc_5.pdf'

In [7]:
raw_texts = list(map(lambda v: v['text_dialog'], data['data']))
raw_time = list(map(lambda v: v['time'].split(',')[0], data['data']))
print(len(raw_texts), len(raw_time))

3483 3483


In [8]:
#agent = AgentConnector.open()
#agent.generate("Сколько будет 2 + 2?")

In [17]:
extractor = LLMExtractor(agent_conn=agent)

In [18]:
extracted_triplets = []

In [19]:
from copy import deepcopy

In [45]:
from tqdm import tqdm_notebook, tqdm

In [21]:
for i in tqdm(range(100)):
    # out = extractor.extract(raw_texts[i])
    dd = deepcopy(data[i])
    dd.pop("text")
    out = extractor.extract(data[i]["text"], hyper_node_prop=dd, simple_rel_prop=dd)
    extracted_triplets.append(out)
    # print(raw_texts[i])
    # print(out)
    # print("===================")

100%|██████████| 100/100 [07:40<00:00,  4.61s/it]


In [12]:
kg_model.graph_db.close()

NameError: name 'kg_model' is not defined

In [22]:
len(out)

78

In [25]:
result = []
for elem in extracted_triplets:
    result += elem 

In [26]:
len(result)

6130

In [27]:
import sys
import json
from tqdm import tqdm
from typing import Dict
import numpy as np
import torch
import os
import gc
from gigachat import GigaChat 
from gigachat.models import Chat, Messages

# TO CHANGE
BASEDIR = os.path.abspath(os.getcwd() + "/../")
__import__('pysqlite3')
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
# TO CHNAGE
os.environ["LD_LIBRARY_PATH"] = BASEDIR + "/lib"
sys.path.insert(0, BASEDIR)

EVAL_DATADIR = BASEDIR + '/debug/diaasq'

from src.qa_pipeline import QAPipeline, QAPipelineConfig
from src.qa_pipeline.query_parser import QueryLLMParser
from src.qa_pipeline.knowledge_comparator import KnowledgeComparator
from src.qa_pipeline.knowledge_retriever import KnowledgeRetriever, KnowledgeRetrieverConfig, AStarGraphSearchConfig
from src.qa_pipeline.answer_generator import QALLMGenerator
from src.agents.utils import SYSTEM_PROMPT

# from src.llm_agent import AgentConnector
from src.knowledge_graph_model import KnowledgeGraphModel
from src.neo4j_functions import Neo4jConnection
from src.embedding_functions import EmbeddingsDatabaseConnection, EmbeddingsDatabaseConnectionConfig, VectorDBConnectionConfig, EmbedderModelConfig

from src.local_graph_connection import LocalGraphConnection
from src.utils.data_structs import Triplet, Node, Relation

from src.utils import ReaderMetrics
from src.embedding_functions import ChromaConnection, VectorDBConnectionConfig, EmbeddingsDatabaseConnectionConfig

In [30]:
graph_connection = LocalGraphConnection()

In [28]:
class GigaChatAgent:
    def __init__(self, creds: str, scope: str = 'GIGACHAT_API_CORP', model: str = "GigaChat-Pro",
                 verify_ssl_certs: bool = False) -> None:
        self.giga_model = GigaChat(
            credentials=creds, scope=scope, verify_ssl_certs=verify_ssl_certs, model=model) 
        self.system_prompt = SYSTEM_PROMPT

    def generate(self, user_prompt: str, assistant_prompt: str = None, 
                 system_prompt: str = None, gen_strategy: Dict = None) -> str:

        chat = Chat(messages=[Messages(role='system', content=system_prompt if system_prompt is not None else self.system_prompt), 
                            Messages(role='user', content=user_prompt)]) 
        response = self.giga_model.chat(chat) 
        return response.choices[0].message.content
    
API_KEY = "OWUwOGUzOWEtMjJiNi00YmMxLThmMmItNzMwNjM2MTI2YmYxOjg2ODdiOTVhLTZkNDctNGFjOC1iMmViLTEyNDA5MmFiN2Q5Mw=="
agent = GigaChatAgent(creds=API_KEY)

In [31]:
kg_model = KnowledgeGraphModel(
    graph_db=graph_connection,
    embeddings_db=EmbeddingsDatabaseConnection(
        config=EmbeddingsDatabaseConnectionConfig(
            nodes_db_config=VectorDBConnectionConfig(
                path="../../data/graph_structures/vectorized_nodes/v8/densedb",
                db_name="vectorized_nodes_2_test",
                is_exist = False
            ),
            triplets_db_config=VectorDBConnectionConfig(
                path="../../data/graph_structures/vectorized_triplets/v4/densedb",
                db_name="vectorized_triplets_2_test",
                is_exist = False
            ),
            embedder_config=EmbedderModelConfig(
                model_name_or_path="/app/distrib/isu-top-question-answer/src/gigasum/multilingual-e5-large"
            )
        )
    )
)

In [33]:
graph_connection.create_triplets(result)

[(7957116925150626168, -5747067123848800892, 2909072414898322862),
 (7957116925150626168, 2951931812858495373, 6960757802458827540),
 (2909072414898322862, 2951931812858495373, 6960757802458827540),
 (6645289634592361249, -4484361770959908197, -460849618431496146),
 (6645289634592361249, 6557575681894952924, -9089146858851985469),
 (6645289634592361249, 348946478957316059, 5103770164238489858),
 (6645289634592361249, -1602712761326740190, 9013572710444554489),
 (6645289634592361249, -4449110230756980013, 8682589693125208178),
 (6645289634592361249, 6723750543882323393, 5082481486651327094),
 (6645289634592361249, -529585443616892297, 5514559543017705972),
 (6645289634592361249, 487588034758661940, 2414005211448655065),
 (6645289634592361249, -8072187356969305257, 8801946661013704473),
 (6645289634592361249, -4273134488941657865, 1633106132292812057),
 (6645289634592361249, -53921684593091052, -1799647911902647637),
 (-6735353814883927365, -53921684593091052, -1799647911902647637),
 (48

In [34]:
all_triplets = [elem for elem in graph_connection.triplets_ids.values()]

In [35]:
all_triplets[0]

Triplet(start_node=Node(name='государственная автоматизированная информационная система «эра-глонасс»', type=<NodeType.object: 'object'>, id=7957116925150626168, prop={}, stringified='государственная автоматизированная информационная система «эра-глонасс»'), relation=Relation(name='является', type=<RelationType.simple: 'simple'>, id=-5747067123848800892, prop={'doc_name': 'doc_5.pdf', 'page': '0'}), end_node=Node(name='государственной системой', type=<NodeType.object: 'object'>, id=2909072414898322862, prop={}, stringified='государственной системой'), id='795711692515062616857470671238488008922909072414898322862', stringified='государственная автоматизированная информационная система «эра-глонасс» является (doc_name: doc_5.pdf; page: 0) государственной системой')

In [43]:
import pickle
pickle.dump(all_triplets, open("./all_triplets.pkl", "wb"))
pickle.dump(graph_connection, open("./graph_connection.pkl", "wb"))
# pickle.dump(kg_model, open("./kg_model.pkl", "wb"))

In [36]:
kg_model.embeddings_db.add_triplets(all_triplets)

  0%|          | 0/41 [00:00<?, ?it/s]


ValueError: Expected each embedding in the embeddings to be a list, got ['ndarray']

In [40]:
test_all_tr = pickle.load(open("./all_triplets.pkl", "rb"))

In [41]:
test_all_tr[0]

Triplet(start_node=Node(name='государственная автоматизированная информационная система «эра-глонасс»', type=<NodeType.object: 'object'>, id=7957116925150626168, prop={}, stringified='государственная автоматизированная информационная система «эра-глонасс»'), relation=Relation(name='является', type=<RelationType.simple: 'simple'>, id=-5747067123848800892, prop={'doc_name': 'doc_5.pdf', 'page': '0'}), end_node=Node(name='государственной системой', type=<NodeType.object: 'object'>, id=2909072414898322862, prop={}, stringified='государственной системой'), id='795711692515062616857470671238488008922909072414898322862', stringified='государственная автоматизированная информационная система «эра-глонасс» является (doc_name: doc_5.pdf; page: 0) государственной системой')